<a href="https://colab.research.google.com/github/rabimba/GDE-ML-Artifacts/blob/main/%5BGemma_1%5DCommon_use_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma - All in One

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies
Run the cell below to install all the required dependencies.

In [ ]:
!pip install -q -U keras keras-nlp

## Exploring prompting capabilities

### Gemma

In [ ]:
import keras
import keras_nlp
from IPython.display import display, Markdown, Latex

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
# Let's load Gemma using Keras
gemma_model_id = "gemma_1.1_instruct_2b_en"
gemma = keras_nlp.models.GemmaCausalLM.from_preset(gemma_model_id)

#### Prompting example: Summarization

In [ ]:
prompt = """<start_of_turn>user

Text:
The Great Salt Lake is one of the saltiest bodies of water in the world, and if you have the option of visiting it for the day I’d highly recommend it!
Because of the water’s salinity, many people compare the floating experience to that in the Dead Sea. If you decide to go swimming in Great Salt Lake, only do so in designated swimming areas–Antelope
State Park has several that feature facilities for visitors.A word of caution: swimming (especially swimming in salt water) can dehydrate your body extremely quickly.
Make sure to limit your time in the water and bring proper methods of rehydration to help your body refresh after a swim.
The salt in the water can also burn any cuts or your eyes, so it’s recommended to avoid full-body submersion.If swimming isn’t your thing, you can still enjoy hiking around Great Salt Lake and Antelope State Park!
Check out this hiking-focused tour that features awesome wildlife viewing!Perhaps one of my favorite arboretums I’ve visited so far is Red Butte Garden and Arboretum, affiliated with the University of
Utah. The diversity of flora present in this garden facility is insane! Not to mention the stunning views of the Salt Lake Valley and the foothills of the Wasatch Range.My personal recommendation is to
pack a light picnic or refreshing beverages to enjoy during your walk around the property. There are several private, quaint sitting spots throughout the gardens, and it’s incredibly tranquil and serene.
I’d also recommend packing a book and sunscreen!If you’re craving those scenic mountain views, take a guided drive through the Wasatch Mountain Range for a few hours!

Could you summarize the text in a few words?<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=512)
display(Markdown(response[len(prompt) :]))

Summary:
Google is a tech giant known for its search engine, email, documents, maps, and more, encompassing various products and services.

#### Prompting example: Reasoning

In [ ]:
prompt = """<start_of_turn>user
Think and write your step-by-step reasoning before responding.

Explain why the sky is blue.<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))

**Step 1: Understand the physics of light scattering.**

- Light is composed of different wavelengths, each with a distinct color.
- When sunlight enters the atmosphere, it is scattered by molecules and particles in the air.


**Step 2: Explain the blue wavelength's scattering effect.**

- Blue light has a shorter wavelength than other colors in the spectrum.
- When blue light interacts with molecules in the atmosphere, it is scattered more strongly than other colors.


**Step 3: Describe the scattering process.**

- Blue light is scattered more efficiently by nitrogen molecules than by oxygen molecules.
- This means that more blue light reaches our eyes from the sky than other colors.


**Step 4: Quantify the effect.**

- The blue sky effect is most pronounced on a clear, sunny day with a high concentration of nitrogen molecules in the atmosphere.
- Studies have shown that the blue sky effect accounts for about 47% of the visible light that reaches our eyes from the sky.


**Step 5: Conclusion.**

- The blue sky is caused by the scattering of blue light by molecules in the atmosphere.
- This scattering effect is responsible for the blue color we perceive in the sky.

#### Prompting example: Classification

In [ ]:
prompt = """<start_of_turn>user
Classify the text into neutral, negative, or positive.
Generate only the class, nothing else.
Text: Text: I think the food was awesome.<end_of_turn>
<start_of_turn>model
Sentiment:"""

response = gemma.generate(prompt, max_length=64)
print("Sentiment: " + response[len(prompt) :])

Sentiment:  Positive


#### Prompting example: Extraction

In [ ]:
text = """Lost in a worn Berlin travel guide, Ania underlined Warsaw and London.
Two cities, two dreams: a bustling career in finance and
a cozy bookshop tucked away in a cobbled London street."""

prompt = f"""<start_of_turn>user
Your task is to find all the city names in the given text.
Your response is an array of the model names in the format [\"city_name\"].
If you don't find model names in the abstract or you are not sure, return [\"\"]

Text:
{text}<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=512)
print("Extracted cities: " + response[len(prompt) :])

Extracted cities: ["Berlin", "Warsaw", "London"]


#### Prompting example: Multi-turn dialogue

In [ ]:
import textwrap


def format_messages(messages):
    fmt_messages = [
        f"<start_of_turn>{author}\n{text}<end_of_turn>" for author, text in messages
    ]
    fmt_messages.append("<start_of_turn>model")
    return "\n".join(fmt_messages)


history = [
    (
        "user",
        "You're a helpful assistant. Answer user's question in a friendly and helpful way. "
        "Please introduce yourself first.",
    )
]

print("\n*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***\n")
while True:
    prompt = format_messages(history)
    response = gemma.generate(prompt, max_length=512)
    response = response[len(prompt) :]

    history.append(("model", response))
    print(textwrap.fill(f"Gemma: {response} \n", 120))

    question = input("User: ")
    if question == "EXIT":
        break

    print()
    history.append(("user", question))


*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***

Gemma:  Hi there! I'm your friendly AI assistant here to help you with any questions or tasks you may have. I'm here to
assist you with information, provide solutions, and make your experience more efficient. Let's get started!
User: What can you do?

Gemma: I can assist you with:  * Providing information and answering questions on various topics * Generating creative
content and writing * Translating text and documents * Scheduling appointments and managing tasks * Providing
recommendations and suggestions * And much more!  How can I help you today?
User: EXIT


#### Prompting example: Code Generation
_Note: While Gemma offers some basic code generation capabilities, a more specialized model called CodeGemma is introduced in the next section for these tasks._

In [ ]:
prompt = """<start_of_turn>user
Generate a Python function that multiplies two numbers<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))

```python
def multiply(num1, num2):
  """
  This function multiplies two numbers.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The product of num1 and num2.
  """

  product = num1 * num2
  return product
```

**Example Usage:**

```python
# Multiply 5 and 3
product = multiply(5, 3)

# Print the product
print(product)  # Output: 15
```

**Explanation:**

* The function takes two arguments, `num1` and `num2`.
* Inside the function, we use the multiplication operator (`*`) to calculate the product of `num1` and `num2`.
* The product is stored in the `product` variable.
* The function returns the calculated product.

**Additional Notes:**

* This function only works for positive numbers. For negative numbers, a `ZeroDivisionError` will be raised.
* The function is designed to be simple and efficient. It does not handle errors or special cases.

In [ ]:
# We need to restart the session to release GPU memory
# before loading different model
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}